# Creating an app

# in 5 (not so simple) steps 


1. Setting up our **Repo** 😺
2. Creating a Dummy **App** 💩
3. Backend: setting an **API** 🤖
4. Deploying our Backend as a **Container** 📦
5. Frontend: **Streamlit** 🪄


## 1. Setting up our Repo 😺

In our the terminal, we create a folder for the new Repo

In [1]:
mkdir dummyproject

Move inside the new project folder and initialize Git

In [ ]:
cd dummyproject
git init

If you see `(main)` appearing after your working directory, we are good for the next step.

in case your branch is named (master) you can run this command

In [ ]:
rm -f -r .git #deletes git repo
git config --global init.defaultBranch main #sets main as default

we create a repo using github cli, from our terminal

In [ ]:
gh repo create dummyapp --public


Now our new repository is connected to github 🎉

guidelines for choosing a repo name:
  - use lowercase
  - avoid underscores `_`
  - do not use hyphenes `-`
  - be short but meaningful


## 2. Creating a Dummy **App** 💩

### The files we need for our bare minimun architecture 🩻

`setup.py`
Here we specify our package name and all dependecies (requirements.txt)
this is necesary to be able to install our package with `pip install .`

In [ ]:
from setuptools import find_packages
from setuptools import setup

with open("requirements.txt") as f:
    content = f.readlines()
requirements = [x.strip() for x in content if "git+" not in x]

setup(name='dummyapp',
      description="example dummy package",
      license="MIT",
      author="me",
      install_requires=requirements,
      packages=find_packages(),
      include_package_data=True,
      zip_safe=False)

`requirements.txt`
is the file that is going to serve all the python libraries that we are going to need to run our package
for this example we are going to only two

In [ ]:
fastapi
uvicorn

`.env`
this file contains enviroment variables that are used across the app, but also contains sensible data that needs to be protected like passwords and apikeys, this file is never going to be push to github. 

In [ ]:
#example
GCP_REGION=europe-west1
GAR_REPO=dummyproject
PORT=8000

`.gitignore`
we need this file before committing to github, it is the one that will prevent us from pushing any unnecessary file, for example: our precious `.env` file, too big files, or cache files.

In [ ]:
.env
.env.yaml

# OS generated files #
.DS_Store
__MACOSX
# Jupiter Notebook #
.pytest_cache
__pycache__
*checkpoint.ipynb
*.ipynb_checkpoints

# inteliJ #
.idea
*.gz
*PKG-INFO
*.egg-info
*.csv
*.zip
# IDE #
*.vscode

Finally we need to create the python files and folders.
Let's divide our code into backend and frontend.

```
├── 📂 dummyapp
│   └── 🐍 api.py
├── 📂 frontend
│   └── 🐍 streamlit-app.py
├── .env
├── .gitignore
├── 📄 requirements.txt
└── 🐍 setup.py
```

There are other secondary files such as
`.python-version` this file is created when we configure our virtual environment.
`Makefile` this file is used to simplify our terminal commands (it is a great hack!).

Our last step, we configure the vitual environment

In [ ]:
pyenv virtualenv 3.10.6 dummyproject-env
pyenv local dummyproject-env

And finally you have to install our package! 🎉
we need this command to let python know the relationship between our files.

`pip install -e . `

`e` means developer mode, all changes take effect without reinstalling the package.
`.` means, we are referring to our current path.

We are ready now for our first push to github

In [ ]:
git add .
git commit -m "Initial commit"
git push origin main

## 3. Backend: setting an **API** 🤖

We are going to code a really simple python script...
### let's get creative ✨

we just need to remember the basic structure of a python file

- imports
  - import request
  - import os
- main or body
  - our code with our class or functions
- entry point
  - signalized by `if __name__ == "__main__":` 

When we are sure that our functions are working locally, we can set up our API

**fastApi to the rescue!**

In [ ]:
from fastapi import FastAPI

api = FastAPI()

# Define a root `/` endpoint
@api.get('/')
def index():
    return {'status': 'online'}

for executing this code, we just need to launch our server with uvicorn

In [ ]:
uvicorn dummyapp.backend.api:api --reload

Is this command too long to remember but you know you will be using it every time you need to execute your api?

**Makefile to the rescue!**
This way we don't need to remember the entire sintax

In [ ]:
touch Makefile

# Makefile content
install:
    pip install -e .

run_api:
    uvicorn dummyapp.backend.api:api --reload

## 4. Deploying our Backend as a **Container** 📦

### Time for DOCKER! 🐳

First step is to create a `Dockerfile`
it contains the instructions for building our image

In [ ]:
FROM python:3.8.12-buster # base image

WORKDIR /dummyproject # set working directory, create if not exists

COPY requirements.txt requirements.txt # copy requirements file to working directory
RUN pip install -r requirements.txt # install requirements

COPY backend backend # copy backend directory to working directory
COPY setup.py setup.py # copy setup file to working directory
RUN pip install . # install package

CMD uvicorn dummyapp.backend.api:api --host 0.0.0.0 --port $PORT # run the API

We need then to execute quite some commands in the terminal
why we don't use our make file then? 😎

[docker visualized](https://docs.google.com/presentation/d/1IrYWAfcgdYL29petZOd_xJKql8U3ylenLRjZXFAOC5Q/edit?usp=sharing)

In [ ]:
############### build and run locally ####################3
build-local:
	docker build -t dummyapp:dev .

run-local:
	docker run -p 8000:8000 --env-file .env dummyapp:dev

run-interactive:
	docker run -it -p 8000:8000 --env-file .env dummyapp:dev sh

We now tested that our code is running perfectly inside a container.
It's time to **deploy to the cloud** 

In [ ]:
############## register and authorize Gcloud Artifact Registry ############3
gcloud-auth:
	gcloud auth configure-docker ${GCP_REGION}-docker.pkg.dev

gcloud-register-artifact:
	gcloud artifacts repositories create ${GAR_REPO} --repository-format=docker \
	--location=${GCP_REGION} --description="Repository for storing dummyapp images"

############# building and pushing to Gcloud Artifact Registry ############3
build-gcloud:
	docker build -t ${GCP_REGION}-docker.pkg.dev/${GCP_PROJECT}/${GAR_REPO}/${GAR_IMAGE}:prod .

build-test-gcloud:
	docker run -p 8000:8000 --env-file .env ${GCP_REGION}-docker.pkg.dev/${GCP_PROJECT}/${GAR_REPO}/${GAR_IMAGE}:prod

push:
	docker push ${GCP_REGION}-docker.pkg.dev/${GCP_PROJECT}/${GAR_REPO}/${GAR_IMAGE}:prod

############# deploy to Google cloud Run #################################3
deploy:
	gcloud run deploy --image ${GCP_REGION}-docker.pkg.dev/${GCP_PROJECT}/${GAR_REPO}/${GAR_IMAGE}:prod \
	--memory ${GAR_MEMORY} --region ${GCP_REGION} --env-vars-file .env.yaml --allow-unauthenticated

############ manage Google Cloud Run #####################################3
status:
	gcloud run services list

stop-gcloud:
	gcloud run services delete ${GAR_IMAGE}

# 🤯

right?

don't worry is all in the Makefile

Now our app is online and serving traffic, 
you should be able to see a public url in your terminal.

It's possible that you need to authorize traffic first, this we need to set up in Google Cloud Run

Remember:
- GCP stands for Google Cloud Platform
- GAR stands for Google Artifact Repository

# 5. Frontend: **Streamlit** 🪄

The logic of streamlit is similar to a print

In [ ]:
import streamlit as st

sometext = 'Hello World'

st.write(sometext) # this is shown in the webpage
print(sometext) # this is shown in the terminal


Let's keep experimenting on our dummyapp 🤗

Don't Forget to check this amazing resource to play and experiment with streamlit
# [streamlit le wagon resource](https://streamlit.lewagon.ai)

```
import streamlit as st
st.write('Enjoy today's challenges! 🤗')

streamlit run thank.you!
```
